# Python Design Patterns II
- The adapter pattern 
- The facade pattern 
- Lazy initialization and the flyweight pattern 
- The command pattern 
- The abstract factory pattern 
- The compositon pattern 

__By Will Norris__

## The Adapter Pattern 
- Adapters are used to allow two pre-existing objects to work together, even if their interfaces are not compatible 

__What is it good for?__ 
- Converting arguments to a different format 
- Rearranging the order of arguments 
- Calling a differently named method 
- Supplying default arguments 

__Essentially__ they are just the "connection" that links two different objects (like dongles for our computers) 

#### Structure: 
- Similar to simplified decorator pattern
- Map between two different interaces 
    - Whereas decorators provide the same interface they replace 
 

![Adapter Patterns- SOURCE](https://i.imgur.com/qkyXVp8.png)

- ```Interface1``` is expecting to call a method called ```make_action(some, arguments)```
- We already have ```Interface2``` class that does the things we want 
    - However, it provides a method called ```different_actin(other, arguments)``` instead!
- The ```Adapter``` class implements the ```make_action``` interface and maps the arguments to the existing interface. 


### What's the Advantage to Adapters: 
- Code that maps from one interface to another is all in one place 
    - If we didn't do this we would have to convert whenever we wanted to use that code!!

In [1]:
class AgeCalculator:
    def __init__(self, birthday):
        # this class formats the year month, day into int's from strings
        self.year, self.month, self.day = (
            int(x) for x in birthday.split('-'))
        
    def calculate_age(self,date):
        year, month, day = (
                int(x) for x in date.split('-'))
        age = year - self.year
        if (month,day) < (self.month,self.day):
            age -= 1
        return age 

### What's Wrong Here? 
- Class takes in a string for a birthday
    - Many pythonistas 🐍 might expect to input a datetime object instead 
    - Datetime objects are far more flexible and easier to build on top of later!!

__Solutions__: 
1. We could __rewrite the class__ to accept datetime ☀️
    - not always an option! ☹️
2. We could just __convert strings to datetime on the fly__ with ```datetime.date.strftime('%Y-%m-%d')``` ⛈
    - But honestly, do you want to do that over and over again? 
    - Leaves you exposed to mistypes ('%M') is minutes whereas ('%m') is months!
3. We could __write an adapter__ that allows a normal date to be entered into a normal ```AgeCalculator``` class 🌤

In [2]:
import datetime
class DateAgeAdapter:
    def _str_date(self, date):
        return date.strftime("%Y-%m-%d")
    
    def __init__(self, birthday):
        birthday = self._str_date(birthday)
        self.calculator = AgeCalculator(birthday)
        
    def get_age(self, date): # method name can be different, and arguments can change
        date = self._str_date(date)
        return self.calculator.calculate_age(date)

Now we can enter datetime objects instead of strings but use our existing codebase:

In [3]:
d = DateAgeAdapter(datetime.date(1975,1,1))
print("Adapter Implementation: {}".format(d.get_age(datetime.date.today())))

d1 = AgeCalculator("1975-01-01")
print("String Implementation: {}".format(d1.calculate_age("2019-03-17")))

Adapter Implementation: 44
String Implementation: 44


### Downsides to Class Adapaters: 
- Over time we forget why we need this interface unless it's really clear 
- Unless there is significant downsides to restructuring the original code we don't want to keep adding more adapters to keep  old classes useable
    - More classes = more overhead
__Alternatives to Classes__:
- We can often monkey patch at runtime 
- WE can often times use a simple function as an adapter 

## The Facade Pattern: 
- Designed to provide a simple interface to a complex system of components.
    - If a system is really complex, we don't usually want to interact with it directly 
    - We often have a set of "typical" use cases 
- __Facade patterns allow us to create a set of simplified operations to simplify your interaction with complex objects without compromising the ability to interact directly__

![](https://imgur.com/Y6bA3K3.png)

### Facade vs Adapter 
- Facade trying to abstract a simpler inferface from a complex one
- Adapter is trying to map one existing interface to another

### Now, an Example:
Let's create a facade for an email application. 
- There are two libraries for receiving email messages,```smtplib```/```imaplib```, and they are quite complex!
- Let's write a facade that allows us to send an email and list the emails in our inbox on either an ```IMAP``` or ```SMTP``` connection.

__Structure:__
- Makes assumptions about the connection 
    - Host for SMTP and IMAP are same
    - Username and password for each is same 
    - They both use standard ports 

This will cover most email needs, but nothing is preventing a programming from cracking open the hood and accessing the core functionality


In [4]:
import smtplib
import smtplib

class EmailFacade:
    def __init__(self, host, username, password):
        self.host = host
        self.username = username
        self.password = password
    
    def send_email(self, to_email, subject, message):
        if not "@" in self.username: # is username wino6687 or wino6687@colorado.edu
            from_email = "{0}@{1}".format(
                    self.username, self.host)
        else:
            from_email = self.username
        message = ("From: {0}\r\n"
                  "To: {1}\r\n"
                  "Subject: {2}\r\n\r\n{3}").format(
                            from_email,
                            to_email,
                            subject,
                            message)
        
        smtp = smtplib.SMTP(self.host)
        smtp.login(self.username, self.password)
        smtp.sendmail(from_email, [to_email],message)    

In [5]:
def get_inbox(self):
    mailbox = imap.IMAP4(self.host)
    mailbox.login(bytes(self.username, 'utf8'),
                 bytes(self.password, 'utf8'))
    mailbox.select()
    x,data = mailbox.search(None, 'ALL')
    messages = []
    for num in data[0].split():
        x,message = mailbox.fetch(num, '(RFC822)')
        messages.append(message[0][1])
    return messages

- Now we can send and receive email relatively simply
    - Easily abstracting away the complexity we don't need 
- This is a very pythonic thing to do 
    - The python language as whole likes to offer abstractions to make your life esier (for loop structure, list comprehensions, generators) 

## The Flyweight Pattern
- Memory optimization pattern 
- It is tempting to ignore memory usage and allow Python's wonderful garbage collector to do all of our work for us 🚛
    - But __as things grow, paying attention to memory can be critical!__ Especially if you are working on a local machine or limited cloud resources
    
__Concept__:
- ensures that objects that share a state can use the same memory for that shared state. 
- Often implemented after a program has demonstrated memory problems 
    - Don't have to start with this in mind 
    - __"Premature optimization is the most effective way to create a program that is too complicated to maintain"__

![flyweight](https://imgur.com/NRGX2uu.png)

- Each ```Flyweight``` has no specific state; any time it needs to perform an operation on ```SpecificState```, that state needs to be passed into the ```Flyweight``` by the calling code
- The factory that returns a flyweight is a seperate object
    - it's purpose is to return a flyweight for a given key identifying that flyweight. 
    - If the flyweight exists, we return it; if not, we create a new one

-- finish flyweight with example 

### The Command Pattern: 
- Adds a level of abstraction between actions that must be done, and the object that invokes those actions, normally at a later time.
![](https://imgur.com/uP6s72c.png)



![](https://imgur.com/uP6s72c.png)

- We have client code that creates a ```Command``` object that can be executed at a later date. 
- This object knows about a receiver object that manages its own internal state when the command is executed on it 
- The ```Command``` object implements a specific interface (typically it has an ```execute``` or ```do_action``` method, and also keeps track of any arguments required to perform the action 
- Lastly, one or more Invoker objects execute the command at the correct time 

### Common Use for Command Pattern: Graphical Windows
- Buttons that exist in a simple GUI are essentially just functionility that is built and ready but waiting to be executed 
    - Executing something when it is selected is called __invoking__
- Actions that occur when you select a menu item, a keyboard shortcut, or a toolbar icon are all examples of invoker objects!

__Example:__ Simple command pattern to provide ```Save``` and ```Exit``` actions 

In [8]:
import sys 

class Window:
    def exit(self):
        sys.exit(0)

class Document:
    def __init__(self,filename):
        self.filename = filename 
        self.contents = "This file cannot be modified"
    def save(self):
        with open(self.filename, 'w') as file:
            file.write(self.contents)

__Note__: 
- These objects are trivially simple. 
- In a real environment the Window would need to handle things like mouse clicks and movement. The document would need to handle things like insertion and deletion. 

In [13]:
# define invoker classes
class ToolbarButton:
    def __init__(self,name, iconname):
        self.name = name 
        self.iconname = iconname
    
    def click(self):
        self.command.execute()
    
class MenuItem:
    def __init__(self, menu_name, menuitem_name):
        self.menu = menu_name 
        self.item = menuitem_name
        
    def click(self):
        self.command.execute()

class KeyboardShortcut:
    def __init__(self, key, modifier):
        self.key = key 
        self.modifier = modifier 
        
    def keypress(self):
        self.command.execute()

In [11]:
# hook up the commands 
class SaveCommand:
    def __init__(self, document):
        self.document = document 
    
    def execute(self):
        self.document.save()
        
class ExitCommand:
    def __init__(self, window):
        self.window = window
    
    def execute(self):
        self.window.exit()

In [12]:
window = Window()
document = Document("a_document.txt")
save = SaveCommand(document)
exit = ExitCommand(window)

save_button = ToolbarButton('save', 'save.png')
save_button.command = save
save_keystroke = KeyboardShortcut("s", "ctrl")
save_keystroke.command = save
exit_menu = MenuItem("File", "Exit")
exit_menu.command = exit

In [ ]:
-- finish explaining what this does 

## The Abstract Factory Pattern: 